# Install requirements

In [ ]:
# Install requirements
!pip install fastapi
!pip install opencv-python
!pip install Pillow
!pip install timm
!pip install python-multipart
!pip install uvicorn

# !pip show timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [ ]:
!pip install nest-asyncio
# !pip show timm

In [ ]:
!pip install pyngrok
# !pip show timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 42.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=87fd056643c40f1214d78b17d0c5b6232743ae3720849d872fcd669085155591
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import cv2
import io
import numpy as np

import torch
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

# !pip show timm

## DPT Model

In [ ]:
def load_model_dpt(model_type):
  ## Load model

  midas = torch.hub.load("intel-isl/MiDaS", model_type)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  midas.to(device)
  midas.eval()

  return midas


def pre_process_dpt(image, model_type):

    midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
    if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
        transform = midas_transforms.dpt_transform
    else:
        transform = midas_transforms.small_transfor

    # Load image
    img = cv2.imdecode(np.frombuffer(image.file.read(),
                                      np.uint8),
                        cv2.IMREAD_COLOR)

    # convert it to the correct format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Transform it so that it can be used by the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_batch = transform(img).to(device)

    # Return this image so it can be used in postprocessing
    return input_batch, img

def post_process_dpt(original, prediction):

  prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=original.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

  output = prediction.cpu().numpy()
  # Create a figure using matplotlib which super-imposes the original
  # image and the prediction

  fig = Figure()
  canvas = FigureCanvas(fig)
  ax = fig.gca()

  # Render both images original as foreground
  ax.imshow(original)
  ax.imshow(output)

  ax.axis("off")
  canvas.draw()

  # Reshape output to be a numpy array
  width, height = fig.get_size_inches() * fig.get_dpi()
  width = int(width)
  height = int(height)
  output_image = np.frombuffer(canvas.tostring_rgb(),
                                dtype='uint8').reshape(height, width, 3)

  # Encode to png
  res, im_png = cv2.imencode(".png", output_image)

  return im_png

# !pip show timm


In [ ]:
!pip show timm

Name: timm
Version: 0.9.2
Summary: PyTorch Image Models
Home-page: https://github.com/huggingface/pytorch-image-models
Author: Ross Wightman
Author-email: ross@huggingface.co
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, pyyaml, safetensors, torch, torchvision
Required-by: 


# Setup the server

In [ ]:
# !pip install timm==0.6.13
model_type = "DPT_Large"
model = load_model_dpt(model_type)

# Code from: https://fastapi.tiangolo.com/tutorial/request-files/
app = FastAPI()


@app.post("/uploadfiles/")
async def create_upload_files(files: List[UploadFile] = File(...)):
    """ Create API endpoint to send image to and specify
     what type of file it'll take

    :param files: Get image files, defaults to File(...)
    :type files: List[UploadFile], optional
    :return: A list of png images
    :rtype: list(bytes)
    """

    for image in files:

        # Return preprocessed input batch and loaded image
        input_batch, image = pre_process_dpt(image, model_type)

        # Run the model and postpocess the output
        with torch.no_grad():
            prediction = model(input_batch)

        # # Post process and stitch together the two images to return them
        output_image = post_process_dpt(image, prediction)

        return StreamingResponse(io.BytesIO(output_image.tobytes()),
                                 media_type="image/png")


@app.get("/")
async def main():
    """Create a basic home page to upload a file

    :return: HTML for homepage
    :rtype: HTMLResponse
    """

    content = """<body>
          <h3>Upload an image to get it's depth map from the MiDaS model</h3>
          <form action="/uploadfiles/" enctype="multipart/form-data" method="post">
              <input name="files" type="file" multiple>
              <input type="submit">
          </form>
      </body>
      """
    return HTMLResponse(content=content)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v3/dpt_large_384.pt" to /root/.cache/torch/hub/checkpoints/dpt_large_384.pt
100%|██████████| 1.28G/1.28G [01:19<00:00, 17.4MB/s]


In [ ]:
!ngrok config add-authtoken 2SVOcAUAvBqGEo50Mc035SEdyIU_DMHingLrdYSifWjb5FWy

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok authtoken 2SVOcAUAvBqGEo50Mc035SEdyIU_DMHingLrdYSifWjb5FWy

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import subprocess

auth_token = "2SVOcAUAvBqGEo50Mc035SEdyIU_DMHingLrdYSifWjb5FWy"

# Run the ngrok authentication command
command = f"ngrok authtoken {auth_token}"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, error = process.communicate()

# Check if any error occurred during authentication
if error:
    print("Error occurred during authentication:")
    print(error.decode("utf-8"))
else:
    print("Ngrok authentication successful!")


Ngrok authentication successful!


In [ ]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, bind_tls=True, proto="http")

# Print the public URL
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://1776-34-143-212-154.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
# Check if it exists
!ps aux | grep ngrok

root        1430 19.0  0.2 736704 27764 ?        Sl   09:03   0:00 /usr/local/lib/python3.10/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root        1444  0.0  0.0   6904  3188 ?        S    09:03   0:00 /bin/bash -c ps aux | grep ngrok
root        1446  0.0  0.0   6444   652 ?        S    09:03   0:00 grep ngrok


# Make magic happen

In [ ]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
print(public_url)
uvicorn.run(app)

INFO:     Started server process [543]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


NgrokTunnel: "https://1776-34-143-212-154.ngrok-free.app" -> "http://localhost:8000"
INFO:     2402:e280:21c4:129:a4c5:2bb4:63d4:63ff:0 - "GET / HTTP/1.1" 200 OK
INFO:     2402:e280:21c4:129:a4c5:2bb4:63d4:63ff:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2402:e280:21c4:129:a4c5:2bb4:63d4:63ff:0 - "GET / HTTP/1.1" 200 OK


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


INFO:     2402:e280:21c4:129:a4c5:2bb4:63d4:63ff:0 - "POST /uploadfiles/ HTTP/1.1" 200 OK
INFO:     2402:e280:21c4:129:a4c5:2bb4:63d4:63ff:0 - "GET /uploadfiles/ HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [543]


In [ ]:
# Kill tunnel
ngrok.disconnect(public_url=public_url)